In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/new_hack.csv')

In [4]:
clip_lables = df['clip_lables'].unique().tolist()
clip_lables

['Спортивное сооружение Динамо в Екатеринбург где Здание арафед с синим и белым внешним видом и синим забором',
 'Спортивное сооружение Динамо в Екатеринбург где Вид на воздушный вид на город с футбольным полем и большим апельсиновым куполом',
 'Спортивное сооружение Динамо в Екатеринбург где вид на здание с часовой башней на вершине',
 'Спортивное сооружение Динамо в Екатеринбург где Здание арафед с синим и белым знаком на нем',
 'Спортивное сооружение Динамо в Екатеринбург где Арафед здание с синим и белым знаком на его стороне',
 'Спортивное сооружение Динамо в Екатеринбург где вид на город с большим зданием и большим куполом',
 'Спортивное сооружение Динамо в Екатеринбург где Взгляд на город с футбольным полем и стадионом',
 'Спортивное сооружение Динамо в Екатеринбург где Взгляд на футбольное поле и реку в городе',
 'Спортивное сооружение Динамо в Екатеринбург где Вид на здание с башней посреди снежной зоны',
 'Спортивное сооружение Динамо в Екатеринбург где здание арафед с синим 

In [5]:
# !pip install ruclip==0.0.2

In [6]:
# !pip install huggingface_hub==0.25.00

In [7]:
#@title Allowed Resources
import multiprocessing
import torch
from psutil import virtual_memory

ram_gb = round(virtual_memory().total / 1024**3, 1)

print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device.type)


!nvidia-smi

CPU: 12
RAM GB: 31.8
PyTorch version: 2.2.2
CUDA version: 12.1
cuDNN version: 8801
device: cuda
Tue Dec 17 22:18:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070      WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P5             19W /  115W |     882MiB /   8192MiB |     22%      Default |
|                                         | 

In [8]:
import ruclip
import warnings
import torch
warnings.filterwarnings("ignore", category=UserWarning)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip, processor = ruclip.load('ruclip-vit-base-patch32-384', device=device)

C:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
C:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


In [10]:
device

'cuda'

#### Пример с 1 фото через фото


In [12]:
from PIL import Image
from io import BytesIO
import base64
import PIL
image = [Image.open("data/conserv.jpg")]

In [13]:
predictor = ruclip.Predictor(clip, processor, device, bs=8)
with torch.no_grad():
    text_latents = predictor.get_text_latents(clip_lables)
    pred_labels = predictor.run(image, text_latents)

1it [00:00,  7.20it/s]


In [14]:
df[df['clip_lables'] == clip_lables[pred_labels[0]]]

,Name,WikiData,City,Lon,Lat,description,image,en_txt,category,ru_txt,clip_lables
359,Уральская государственная консерватория,Q4476712,Екатеринбург,60.599995,56.837444,высшее учебное заведение в Екатеринбурге,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,a close up of a building with a clock on it,Учебное заведение,Крупный план здания с часами на нем,Учебное заведение Уральская государственная ко...


#### Пример с фото через base64


In [16]:
elems = df.sample(1)
elems

,Name,WikiData,City,Lon,Lat,description,image,en_txt,category,ru_txt,clip_lables
1807,Рождественская церковь,Q1574846,Нижний Новгород,43.984966,56.327251,Церковь в Нижнем Новгороде,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,a close up of a building with a clock on top o...,Храм/Церковь,Крупный план здания с часами на вершине,ХрамЦерковь Рождественская церковь в Нижний Но...


In [17]:
image = elems['image'].tolist()

In [18]:
from PIL import Image
from io import BytesIO
import base64
import PIL
images = [Image.open(BytesIO(base64.b64decode(image[0])))]

In [19]:
%%time
predictor = ruclip.Predictor(clip, processor, device, bs=8)
with torch.no_grad():
    text_latents = predictor.get_text_latents(clip_lables)
    pred_labels = predictor.run(images, text_latents)

1it [00:00, 43.93it/s]

CPU times: total: 23.9 s
Wall time: 1min 3s


In [20]:
pred_labels[0]

4183

In [21]:
df[df['clip_lables'] == clip_lables[pred_labels[0]]]

,Name,WikiData,City,Lon,Lat,description,image,en_txt,category,ru_txt,clip_lables
4735,Храм Зосимы и Савватия,Q16715602,Ярославль,39.903271,57.636955,храм в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,snowy landscape of a city with a church and a ...,Храм/Церковь,снежный пейзаж города с церковью и рекой,ХрамЦерковь Храм Зосимы и Савватия в Ярославль...


### Обобщённая функция

In [23]:
import ruclip
import warnings
import torch
from PIL import Image
from io import BytesIO
import base64

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [24]:
def predict_image_label(image_path_or_base64, clip_labels, df, model_name='ruclip-vit-base-patch32-384', device=None, is_base64 = False):
    """
    Функция для предсказания метки изображения с использованием CLIP модели.

    Args:
        image_path_or_base64 (string): Путь к изображению или строка base64-кода.
        base64 (bool): Значение показывающее ккая фотография пришла.
        clip_labels (list): Список текстовых меток для классификации.
        model_name (string): Название модели CLIP.
        df (DataFrame): датафрейм где искать найденную строку соответсвия.
        device (string): Устройство ("cuda" или "cpu"). Если не указано, выбирается автоматически.
    Returns: 
        int: Предсказанная текстовая метка.
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    clip, processor = ruclip.load(model_name, device=device)

    if is_base64:
        image_data = base64.b64decode(image_path_or_base64)
        image = Image.open(BytesIO(image_data))
    else:
        image = Image.open(image_path_or_base64)

    predictor = ruclip.Predictor(clip, processor, device, bs=8)

    with torch.no_grad():
        text_latents = predictor.get_text_latents(clip_labels)
        pred_labels = predictor.run([image], text_latents)

    return (df[df['clip_lables'] == clip_lables[pred_labels[0]]]).index[0]

In [25]:
image_path = "data/conserv.jpg"

predicted_label = predict_image_label(image_path, clip_lables, df)

1it [00:00, 40.49it/s]


In [26]:
display(df.loc[predicted_label])

Name                     Уральская государственная консерватория
WikiData                                                Q4476712
City                                                Екатеринбург
Lon                                                    60.599995
Lat                                                    56.837444
description             высшее учебное заведение в Екатеринбурге
image          /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
en_txt               a close up of a building with a clock on it
category                                       Учебное заведение
ru_txt                       Крупный план здания с часами на нем
clip_lables    Учебное заведение Уральская государственная ко...
Name: 359, dtype: object

In [27]:
elems = df.sample(1)
elems

,Name,WikiData,City,Lon,Lat,description,image,en_txt,category,ru_txt,clip_lables
4648,Федоровский собор,Q4494153,Ярославль,39.865444,57.609676,Православный храм в Ярославле,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...,arafed white building with green roof and gree...,Храм/Церковь,Белое здание Arafed с зеленой крышей и зеленым...,ХрамЦерковь Федоровский собор в Ярославль где ...


In [28]:
image = elems['image'].iloc[0]
image

'...'

In [29]:
predicted_label_2 = predict_image_label(image, clip_lables, df, is_base64 = True)


1it [00:00, 53.65it/s]


In [30]:
display(df.loc[predicted_label_2])

Name                                 Церковь Вознесения Господня
WikiData                                                Q4114893
City                                                   Ярославль
Lon                                                    39.871132
Lat                                                    57.626507
description                        Вознесенско-Сретенский приход
image          /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
en_txt         snowy white building with green domes and a ga...
category                                            Храм/Церковь
ru_txt         белоснежное здание с зелеными куполами и ворот...
clip_lables    ХрамЦерковь Церковь Вознесения Господня в Ярос...
Name: 4867, dtype: object

In [31]:
import base64
from io import BytesIO
import torch
from PIL import Image
import ruclip
import pickle

class ClipPredictor:
    def __init__(self, clip_labels, model_name='ruclip-vit-base-patch32-384', device=None):
        """
        Инициализирует предсказатель CLIP с предзагрузкой модели и латентов текстовых меток.

        Args:
            clip_labels (list): Список текстовых меток для классификации.
            model_name (string): Название модели CLIP.
            device (string): Устройство ("cuda" или "cpu"). Если не указано, выбирается автоматически.
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.clip, self.processor = ruclip.load(model_name, device=self.device)
        self.predictor = ruclip.Predictor(self.clip, self.processor, self.device, bs=8)
        self.clip_labels = clip_labels
        self.text_latents = self._compute_text_latents()

    def _compute_text_latents(self):
        """Предварительно вычисляет латенты текстовых меток."""
        with torch.no_grad():
            return self.predictor.get_text_latents(self.clip_labels)

    def predict(self, image_path_or_base64, df, is_base64=False):
        """
        Предсказывает метку изображения.

        Args:
            image_path_or_base64 (string): Путь к изображению или строка base64-кода.
            df (DataFrame): DataFrame с колонкой 'clip_lables'.
            is_base64 (bool): True, если изображение передается в виде строки base64.

        Returns:
            int: Индекс строки в DataFrame, соответствующей предсказанной метке.
        """
        image = self._load_image(image_path_or_base64, is_base64)

        with torch.no_grad():
            pred_labels = self.predictor.run([image], self.text_latents)

        predicted_label = self.clip_labels[pred_labels[0]]
        return df[df['clip_lables'] == predicted_label].index[0]

    def _load_image(self, image_path_or_base64, is_base64):
        """Загружает изображение из файла или строки base64."""
        if is_base64:
            image_data = base64.b64decode(image_path_or_base64)
            return Image.open(BytesIO(image_data)).convert("RGB")
        return Image.open(image_path_or_base64).convert("RGB")

In [32]:
%%time
predictor = ClipPredictor(clip_lables)
index = predictor.predict("data/conserv.jpg", df)

1it [00:00, 55.56it/s]

CPU times: total: 33.4 s
Wall time: 1min 5s


In [33]:
# !pip install python-multipart

In [34]:
# !pip install fastapi uvicorn pillow

In [35]:
from fastapi import FastAPI, UploadFile, Form
from pydantic import BaseModel
import base64
from io import BytesIO
import torch
from PIL import Image
import ruclip
import pandas as pd

class ClipPredictor:
    def __init__(self, clip_labels, model_name='ruclip-vit-base-patch32-384', device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.clip, self.processor = ruclip.load(model_name, device=self.device)
        self.predictor = ruclip.Predictor(self.clip, self.processor, self.device, bs=8)
        self.clip_labels = clip_labels
        self.text_latents = self._compute_text_latents()

    def _compute_text_latents(self):
        with torch.no_grad():
            return self.predictor.get_text_latents(self.clip_labels)

    def predict(self, image, df):
        with torch.no_grad():
            pred_labels = self.predictor.run([image], self.text_latents)
        predicted_label = self.clip_labels[pred_labels[0]]
        return df[df['clip_lables'] == predicted_label].index[0]

    def load_image(self, image_data: bytes, is_base64=False):
        if is_base64:
            image_data = base64.b64decode(image_data)
        return Image.open(BytesIO(image_data)).convert("RGB")


app = FastAPI()

df = pd.read_csv('data/new_hack.csv')
clip_labels = df['clip_lables'].unique().tolist()
clip_predictor = ClipPredictor(clip_labels)

class PredictionResponse(BaseModel):
    predicted_index: int

@app.post("/predict", response_model=PredictionResponse)
async def predict(
    file: UploadFile = None,
    base64_image: str = Form(None),
    is_base64: bool = Form(False)
):
    """
    Эндпоинт для предсказания метки изображения.
    - file: загружаемый файл изображения.
    - base64_image: изображение в формате base64.
    - is_base64: указатель на то, что изображение передается как base64.
    """
    if file and not is_base64:
        image_data = await file.read()
    elif base64_image:
        image_data = base64_image.encode('utf-8')
        is_base64 = True
    else:
        return {"error": "Either file or base64_image must be provided"}

    image = clip_predictor.load_image(image_data, is_base64=is_base64)
    predicted_index = clip_predictor.predict(image, df)
    return {"predicted_index": predicted_index}